In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.utils import shuffle
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit
from functools import reduce
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix


pd.options.display.max_columns = None

# Step 3
# Working create Spark session 
# Initialize Spark session with the spark-excel library
# Stop any existing Spark session 
# "com.crealytics:spark-excel_2.12:0.13.5" was needed as other configs did not work with the version of Excel the files were supplied with.

fileNameCSV = "WA_Fn-UseC_-HR-Employee-Attrition.csv"
sheet_name = 'WA_Fn-UseC_-HR-Employee-Attriti'
#df = pd.read_excel(fileName, header=0, sheet_name=sheet_name)
try:
    # Attempt to read the CSV file into a DataFrame
    df = pd.read_csv(fileNameCSV)
    print(df.head())
except PermissionError:
    print(f"PermissionError: You do not have permission to read the file '{fileNameCSV}'.")
except FileNotFoundError:
    print(f"FileNotFoundError: The file '{fileNameCSV}' does not exist.")
except Exception as e:
    print(f"An error occurred: {e}")



   Age Attrition     BusinessTravel  DailyRate              Department  \
0   41       Yes      Travel_Rarely       1102                   Sales   
1   49        No  Travel_Frequently        279  Research & Development   
2   37       Yes      Travel_Rarely       1373  Research & Development   
3   33        No  Travel_Frequently       1392  Research & Development   
4   27        No      Travel_Rarely        591  Research & Development   

   DistanceFromHome  Education EducationField  EmployeeCount  EmployeeNumber  \
0                 1          2  Life Sciences              1               1   
1                 8          1  Life Sciences              1               2   
2                 2          2          Other              1               4   
3                 3          4  Life Sciences              1               5   
4                 2          1        Medical              1               7   

   EnvironmentSatisfaction  Gender  HourlyRate  JobInvolvement  JobLevel  

In [14]:
#check for data quality
# Check for missing values in the entire dataset
print("Check for missing values (True indicates missing):")
print(df.isna())

# Check for missing values per column
print("\nCount of missing values per column:")
print(df.isna().sum())

# Check if there are missing values anywhere in the dataset
print("\nAre there any missing values in the dataset?")
print(df.isna().any().any())

# Display the data types of each column
print("Data types of the dataset:")
print(df.dtypes)

Check for missing values (True indicates missing):
        Age  Attrition  BusinessTravel  DailyRate  Department  \
0     False      False           False      False       False   
1     False      False           False      False       False   
2     False      False           False      False       False   
3     False      False           False      False       False   
4     False      False           False      False       False   
...     ...        ...             ...        ...         ...   
1465  False      False           False      False       False   
1466  False      False           False      False       False   
1467  False      False           False      False       False   
1468  False      False           False      False       False   
1469  False      False           False      False       False   

      DistanceFromHome  Education  EducationField  EmployeeCount  \
0                False      False           False          False   
1                False      Fals

In [ ]:
#Now convert the text value in file to a number so statics can be generated and save to a excel format
# Sample DataFrame
# Define the mapping dictionary
# Convert text labels to numerical values for the 'Attrition' column
df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0})
df['BusinessTravel'] = df['BusinessTravel'].map({'Non-Travel': 0, 'Travel_Rarely': 1, 'Travel_Frequently': 2})
df['Department'] = df['Department'].map({ 'Sales': 0, 'Research & Development': 1, 'Human Resources': 2})
df['EducationField'] = df['EducationField'].map({ 'Life Sciences': 0, 'Other': 1, 'Human Resources': 2, 'Marketing': 3, 'Medical': 4, 'Technical Degree': 5})
df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})
df['JobRole'] = df['JobRole'].map({ 'Healthcare Representative': 0, 'Human Resources': 1, 'Laboratory Technician': 2, 'Manager': 3, 'Manufacturing Director': 4, 'Research Director': 5, 'Research Scientist': 6, 'Sales Executive': 7, 'Sales Representative': 8})
df['MaritalStatus'] = df['MaritalStatus'].map({'Single': 0, 'Married': 1, 'Divorced': 2})
df['OverTime'] = df['OverTime'].map({'Yes': 1, 'No': 0})
df['Over18'] = df['Over18'].map({'Y': 1, 'N': 0})

# Check for any missing values after mapping
print(df.isnull().sum())
# Handle any missing values
df = df.dropna()
# Check data quality

# Verify that all features are numeric
print(df.dtypes)

# Save the DataFrame as a CSV file
df.to_csv("WholeDataAsNumbers.csv", index=False)
# Select Attrition, BusinessTravel, JobSatisfaction,  and WorkLifeBalance columns
selectedDF = df.iloc[:, [1, 2, 16, 29]]
selectedDF.to_csv("SelectedDataAsNumbers.csv", index=False)

In [ ]:



#-------------------------------------------
#Below we are doing a correlation matrix to see which features are most correlated with the target variable
# Calculate the correlation matrix same as what we did in lab 5
corr_matrix = df.corr()

# Select features with high correlation to the target
corr_target = corr_matrix['Attrition'].abs().sort_values(ascending=False)
print(corr_target)

# Define features and target
X = df.drop(['Attrition'], axis=1)
y = df['Attrition']
#-------------------------------------------
# Below we are doing a Multiple Linear Regression
# Define the independent variables (features)
feauresDF = df[['BusinessTravel', 'JobSatisfaction', 'WorkLifeBalance']]

# Add a constant (intercept) to the model
feaures = sm.add_constant(feauresDF)

# Define the dependent variable (target)
target = df['Attrition']

# Fit the Multiple Linear Regression model
model = sm.OLS(target, feauresDF).fit()

# Display the model summary
print(model.summary())



In [ ]:


#-------------------------------------------
#Below we are doign a confusion matrix to see how well the model is doing
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Create the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Visualize the confusion matrix
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['No Attrition', 'Attrition'], yticklabels=['No Attrition', 'Attrition'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
pivot_table = df.pivot_table(values='Attrition', index='BusinessTravel', columns='OverTime', aggfunc='sum')
print(pivot_table)
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, fmt="d", cmap="YlGnBu")
plt.title('Pivot Chart')
plt.show()

# Create a pivot table with count of Attrition as values
pivot_table = pd.pivot_table(df, index='Attrition', columns='BusinessTravel', values='JobSatisfaction', aggfunc='count', fill_value=0)

# Display the pivot table
print(pivot_table)

# Reset the pivot table index to use it for plotting
pivot_table = pivot_table.reset_index()

# Melt the pivot table to long format for seaborn
pivot_table_melted = pivot_table.melt(id_vars='Attrition', var_name='BusinessTravel', value_name='Count')

# Plot using seaborn
plt.figure(figsize=(10, 6))
sns.barplot(data=pivot_table_melted, x='BusinessTravel', y='Count', hue='Attrition')
plt.title('Count of Attrition by BusinessTravel')
plt.xlabel('BusinessTravel')
plt.ylabel('Count')
plt.show()

# Group by BusinessTravel and Attrition, and count occurrences
grouped = df.groupby(['BusinessTravel', 'Attrition']).size().unstack(fill_value=0)

# Calculate the percentage of attrition for each BusinessTravel category
grouped['Percentage_Attrition'] = grouped[1] / (grouped[1] + grouped[0]) * 100

# Display the result
print(grouped)


# Group by BusinessTravel and Attrition, and count occurrences
grouped = df.groupby(['BusinessTravel', 'Attrition']).size().unstack(fill_value=0)

# Calculate the percentage of attrition for each BusinessTravel category
grouped['Percentage_Attrition'] = grouped[1] / (grouped[1] + grouped[0]) * 100

# Reset the index to use it for plotting
grouped = grouped.reset_index()

# Plot using seaborn
plt.figure(figsize=(10, 6))
sns.barplot(data=grouped, x='BusinessTravel', y='Percentage_Attrition')
plt.title('Percentage of Attrition by BusinessTravel')
plt.xlabel('BusinessTravel')
plt.ylabel('Percentage of Attrition')

# Display values on the bars
for index, row in grouped.iterrows():
    plt.text(index, row['Percentage_Attrition'] + 1, f"{row['Percentage_Attrition']:.1f}%", color='black', ha="center")

plt.show()

In [ ]:
#Gary messing about

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load your dataset (replace 'your_dataset.csv' with the actual path to your file)
df = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')

# Encode categorical variables
le = LabelEncoder()
for column in df.select_dtypes(include=['object']).columns:
    df[column] = le.fit_transform(df[column])

# Define features and target
X = df.drop('Attrition', axis=1)
y = df['Attrition']

from sklearn.feature_selection import SelectKBest, f_classif

# Select the top 10 features
selector = SelectKBest(score_func=f_classif, k=10)
X_new = selector.fit_transform(X, y)

# Get selected feature names
selected_features = X.columns[selector.get_support()]
print("Selected Features:", selected_features)


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state=42)

# Initialize and train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

import matplotlib.pyplot as plt
import seaborn as sns

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['No Attrition', 'Attrition'], yticklabels=['No Attrition', 'Attrition'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

# Load your dataset (replace 'your_dataset.csv' with the actual path to your file)
df = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')

# Encode categorical variables
le = LabelEncoder()
for column in df.select_dtypes(include=['object']).columns:
    df[column] = le.fit_transform(df[column])

# Define features and target
df.drop('StandardHours', axis=1)
X = df.drop('Attrition', axis=1)
y = df['Attrition']

# Feature selection (optional): For simplicity, we'll use all features here
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

def predict_attrition(model, user_input, X_train):
    # Convert user input into a DataFrame
    input_df = pd.DataFrame([user_input], columns=X_train.columns)
    
    # Check for NaN values and replace them with the mean/mode of the training data
    for column in input_df.columns:
        if input_df[column].isna().any():
            if input_df[column].dtype == 'object' or input_df[column].nunique() < 10:  # Categorical
                input_df[column].fillna(X_train[column].mode()[0], inplace=True)
            else:  # Numerical
                input_df[column].fillna(X_train[column].mean(), inplace=True)
    
    # Predict the probability of attrition
    attrition_prob = model.predict_proba(input_df)[0][1]
    
    # Predict attrition (0: No, 1: Yes)
    attrition_pred = model.predict(input_df)[0]
    
    return attrition_pred, attrition_prob

# Example user input (replace with actual user input)
user_input = {
    'Age': 35,
    'BusinessTravel': 1, # Assuming 1 for Travel_Rarely
    'DailyRate': 110,
    'Department': 1, # Assuming 1 for Research & Development
    'DistanceFromHome': 50,
    'Education': 3,
    'EducationField': 2, # Assuming 2 for Other
    'EnvironmentSatisfaction': 4,
    'Gender': 1, # Assuming 1 for Male
    'HourlyRate': 60,
    'JobInvolvement': 1,
    'JobLevel': 2,
    'JobRole': 1, # Assuming 4 for Manufacturing Director
    'JobSatisfaction': 4,
    'MaritalStatus': 3, # Assuming 3 for Divorced
    'MonthlyIncome': 700,
    'MonthlyRate': 1400,
    'NumCompaniesWorked': 9,
    'OverTime': 1, # Assuming 1 for Yes
    'PercentSalaryHike': 5,
    'PerformanceRating': 1,
    'RelationshipSatisfaction': 1,
    #'StandardHours': 80,
    'StockOptionLevel': 1,
    'TotalWorkingYears': 20,
    'TrainingTimesLastYear': 0,
    'WorkLifeBalance': 1,
    'YearsAtCompany': 1,
    'YearsInCurrentRole': 1,
    'YearsSinceLastPromotion': 0,
    'YearsWithCurrManager': 1
}

# Predict attrition using the user input
attrition_pred, attrition_prob = predict_attrition(model, user_input, X_train)

# Display the prediction result
print(f"Predicted Attrition: {'Yes' if attrition_pred == 1 else 'No'}")
print(f"Probability of Attrition: {attrition_prob:.2f}")

# Initialize and train the Decision Tree model
dt_model = DecisionTreeClassifier(max_depth=3, random_state=42)
dt_model.fit(X_train, y_train)

# Visualize the Decision Tree
plt.figure(figsize=(200, 100))
tree.plot_tree(dt_model, feature_names=X.columns, class_names=['No', 'Yes'], filled=True)
plt.title('Decision Tree for Attrition Prediction')
plt.show()

